# Antidiagonal test

This notebook is to test the speed of multiple antidiagonal averaging approaches.
A `m x k` matrix is created where all antidiagonal elements can be random or they can be increments of 1. The increments of 1 scenario is used to test wether the algorithms are calculating the right values.

In [1]:
# Import libraries
import numpy as np
import time

# Settings
m = 15   # Number of rows
k = 2500  # Number of columns
n = k + m - 1   # Number of samples for storing the results
mat_method = 'rands' # Method to generate matrix
                    # 'rand' = Matrix filled with random numbers
                    # 'ints' = Matrix filled with antigiadonals ints incrementing by 1

# Create matrix based on selected method
raw_mat = np.ones((m,k))   # Initialize matrix

if mat_method == 'rands':
    raw_mat = np.random.rand(m,k)
    flipped_mat = np.fliplr(raw_mat)
elif mat_method == 'ints':
    flipped_mat = np.zeros_like(raw_mat)    # Create flipped matrix 
    
    # For each row and column, fill the matrix with the correct integer
    for idx_m in range(m):                  
        for idx_k in range(k): 
            flipped_mat[idx_m,idx_k] = -idx_m + idx_k
    
    # Flip matrix to get the right orientation
    raw_mat = np.fliplr(flipped_mat)


## First method

In this first implementation, I flipped the matrix once, then I calculated each antidiagonal and computed its mean. This mean would then be stored in the corresponding value of the resulting vector. This methos is implemented using functions and methods to show their speed difference.

### Numpy functions implementation


In [2]:
vect1 = np.zeros(n)  # Initialize new vector

time_start1 = time.time()
temp_mat1 = np.fliplr(raw_mat)  # Temporary matrix with original matrix flipped from left to right

for i_diag, k_diag in enumerate(range(-m+1, k)):
    vect1[-i_diag-1] = np.mean(np.diag(temp_mat1, k=k_diag))

time_end1 = time.time()

### Numpy methods implementation

In [3]:
vect2 = np.zeros(n)  # Initialize new vector

time_start2 = time.time()
temp_mat2 = np.fliplr(raw_mat)  # Temporary matrix with original matrix flipped from left to right

for i_diag, k_diag in enumerate(range(-m+1, k)):
    vect2[-i_diag-1] = temp_mat2.diagonal(offset=k_diag).mean()

time_end2 = time.time()

## Second method

The second method consists on generating a temporary matrix where each column has the values of the k<sup>th</sup> diagonal. A second matrix consisting on booleans to determine the indices used for the average computations. This second matrix is needed as the lenght of the diagonals are not the same. Lastly, the mean value of each column is calculated only for the true values of the second matrix.

In [4]:
time_start3 = time.time()

max_diag = (raw_mat.diagonal(offset=0)).size  # Size of longest diagonal
temp_flip = np.fliplr(raw_mat)
temp_mat3 = np.zeros((max_diag,n))               # Empty matrix to store the values
temp_mask = np.zeros((max_diag,n))

for i,k_diag in enumerate(range(-m+1, k)):
    diag_vals = temp_flip.diagonal(offset=k_diag)
    n_diag = diag_vals.size
    temp_mat3[0:n_diag,i] = diag_vals
    temp_mask[0:n_diag,i] = 1

vect3 = np.flip(temp_mat3.mean(axis=0, where=temp_mask.astype(bool)))

time_end3 = time.time()

## Third method

The third method is similar to the second method but instead of an the diagonal values and mask matrices, we use a list where each element of the list consists of the elements in the k<sup>th</sup> diagonal. Then, a second for loop is used to calculate the mean values of each of the list components.

In [5]:
vect4 = np.zeros(n)  # Initialize new vector

time_start4 = time.time()

# Preallocate variables
max_diag = np.size(raw_mat.diagonal(offset=0))  # Size of longest diagonal
temp_list = [None] * n              # Temporary matrix to store results



# Create list where the ith element includes all kth diagonals
for i, k_diag in enumerate(range(-m+1, k)):
    temp_list[i] = flipped_mat.diagonal(offset=k_diag)

# Calculate average for each element on the list
for i in range(n):
    vect4[-i-1] = np.mean(temp_list[i])

time_end4 = time.time()

## Results

### Time results

Measure the time it takes to run each implementation

In [6]:
print("The results are")
print(f"Method 1 with functions = {1e3*(time_end1-time_start1):.2f} msec")
print(f"Method 1 with methods = {1e3*(time_end2-time_start2):.2f} msec")
print(f"Method 2 (array with mask) = {1e3*(time_end3-time_start3):.2f} msec")
print(f"Method 3 (list) = {1e3*(time_end4-time_start4):.2f} msec")

The results are
Method 1 with functions = 20.00 msec
Method 1 with methods = 10.00 msec
Method 2 (array with mask) = 5.00 msec
Method 3 (list) = 13.00 msec


### Implementation results

Make sure that all implementations yield to the same results

In [7]:
print(f"Method 1f == Method 1m: {np.allclose(vect1, vect2)}")
print(f"Method 1f == Method 2: {np.allclose(vect1, vect3)}")
print(f"Method 1f == Method 3: {np.allclose(vect1, vect4)}")

Method 1f == Method 1m: True
Method 1f == Method 2: True
Method 1f == Method 3: True


## Conclusions

From this implementations we can conclude that the second method is a bit faster. However, this speed benefit starts to become irrelevant for large matrix sizes (e.g., 250 channels x 25k data points). The memory resources were not measured in this test.